# Preparation

In [1]:
import datetime as dt
import re
import json
import time
import pandas as pd
import numpy as np

In [2]:
# An arbitrary integer to use as a sampling seed so the sampling is deterministic and can be re-run
RANDOM_STATE = 13440

frame = pd.read_parquet("secrets/sampling-frame.parquet")
strata_targets = pd.read_table("definitions/strata-sample-targets.tsv").set_index("project_group")
strata_targets = strata_targets.astype(int)

# Set sample sizes

In [3]:
# Stratify sample
strata_sizes = frame.groupby(["project_group", "edit_bin"]).size().unstack()

# `strata_targets` has string column names, but `strata_sizes` has categoricals
strata_targets.columns = strata_sizes.columns

**Staggered sampling:** In 2022, we are exploring a survey distribution mechanism which is likely to improve the response rate. As the sampling target calculation is based on the response rate of a particular strata, so that we meet the required sample size eventually, a higher response rate would require less number of editors to be targeted. For the initial distribution of the survey, for the three large wikis, Wikimedia Commons, German Wikipedia and English Wikipedia, we will target 75% of actual targets. In the follow-up sample, we will be calculating the targets based on the response rate of initial sample of 2022, rather than 2021. This will also help in preventing saturating communities from survey participation calls.

In [21]:
stagger_wikis = ['commons', 'dewiki', 'enwiki']
staggered_strata_targets = strata_targets.copy()
staggered_strata_targets.loc[stagger_wikis, :] = staggered_strata_targets.loc[stagger_wikis, :]*0.75
staggered_strata_targets = staggered_strata_targets.apply(np.ceil).astype(int)
staggered_strata_targets

edit_bin,10-29,30-149,150-599,600-1199,1200+
project_group,,,,,
arwiki,25,60,60,29,98
asia_wps,7,18,77,49,130
cee_wps,115,477,596,302,950
commons,87,330,387,198,999
dewiki,52,368,483,250,490
enwiki,402,1006,639,277,494
eswiki,80,312,369,151,361
frwiki,115,522,559,237,642
itwiki,39,227,255,103,326


In [22]:
# The targets must not be larger than the strata, or the sampling will hit an error
excessive_targets = (staggered_strata_targets > strata_sizes)
assert excessive_targets.any(axis=None) == False

In [23]:
# Set target to 0 for any strata with fewer than 10 users,
# to prevent cases where de-anonymization of data would be too easy
strata_sizes = staggered_strata_targets.where(strata_sizes >= 10, other=0)

In [24]:
strata = (
  strata_sizes
  .unstack()
  .reset_index()
  .rename({0: "sample_size"}, axis=1)
  # Ensure columns are ordered in the expected way
  [["project_group", "edit_bin", "sample_size"]]
)

strata.head()

,project_group,edit_bin,sample_size
0,arwiki,10-29,25
1,asia_wps,10-29,0
2,cee_wps,10-29,115
3,commons,10-29,87
4,dewiki,10-29,52


# Pull samples

In [25]:
def sample_stratum(project_group, edit_bin, sample_size):
    """
    returns the sampled users as a data frame
    """
    stratum = frame.query("project_group == @project_group & edit_bin == @edit_bin")
    sample = stratum.sample(n=sample_size, random_state=RANDOM_STATE)
    return sample

f = []

for row in strata.itertuples(index=False):
    f.append(sample_stratum(*row))
    
sampled_users = pd.concat(f)

# Save samples table

In [26]:
sampled_users.to_csv("secrets/sampled_users.tsv", sep="\t")